<a href="https://www.kaggle.com/code/aayushsin7a/kdtree?scriptVersionId=162826890" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install pygeohash
!pip install geohash2
!pip install pyspark

  Preparing metadata (setup.py) ... - done
  Created wheel for pygeohash: filename=pygeohash-1.2.0-py2.py3-none-any.whl size=6152 sha256=1b37669f1c76b9156cd6e5f68481242383a233c1aad0b7eb2ded00b297e97b47
  Stored in directory: /root/.cache/pip/wheels/28/ec/b6/beadf7295a623f528507691fb0d471b50d064ae9bbad420b8f
Successfully built pygeohash
  Preparing metadata (setup.py) ... done
  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15543 sha256=dfbc716fc4d12eeb1147b6bfbd8daa1b187ebe49c05d9ca9ddedbdf2c9c30d6c
  Stored in directory: /root/.cache/pip/wheels/c0/21/8d/fe65503f4f439aef35193e5ec10a14adc945e20ff87eb35895
Successfully built geohash2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425350 sha256=ef92d11de27672eec0a280b04a02c79aaf73fb33a40459f3ebec73ab2ae70d02
  Stored in directory: /root/.cache/pip/wheels

In [2]:
# Import the required library
import numpy as np # linear algebra
import geopandas as gpd
import numpy as np
import pandas as pd
import folium
from scipy.spatial import cKDTree
from shapely.geometry import Point
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Geohash").getOrCreate()
import numpy as np
from pyspark.sql.functions import *
import os
import sys
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from functools import reduce
from pyspark.sql import DataFrame
import pyspark
#sc=pyspark.SparkContext()
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext
#import sys
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import lead, lag, col ,datediff, to_date, lit, size , length , year, month, dayofmonth, from_unixtime, unix_timestamp, desc, asc, rank, min , expr, trim, date_format, avg
import datetime
from datetime import datetime
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.types import IntegerType, DoubleType, StringType, DateType, TimestampType, StructField, NumericType
from pyspark.sql.functions import isnan
from pyspark.sql.functions import udf
from io import BytesIO
#import boto3
import math
from pyspark.sql import Row
#import pyarrow as pa
import pandas as pd
from datetime import datetime, timedelta
#from time import gmtime, strftime ,datetime
from pyspark.sql.functions import concat, col, lit
from pyspark.sql.functions import count, min,max
from pyspark.sql.functions import col, when
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
from pyspark.sql.functions import col, row_number
import geohash2 as pgh
import pygeohash as pgh2
from scipy.spatial import cKDTree
from shapely.geometry import Point
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/14 08:57:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/kaggle/input/fleets/Fleet.csv
/kaggle/input/worldcities-fortelemetry/worldcities.csv
/kaggle/input/telemetrysample/10min_BEE_GEE_data.csv


In [3]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("NearestCitySearch") \
    .getOrCreate()

24/02/14 08:57:46 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
# Read the dataset 
df = spark.read.csv("/kaggle/input/telemetrysample/10min_BEE_GEE_data.csv",inferSchema=True,header=True)

In [5]:
df.show()

+---+-----------------+-------------+-----------+------------+----------------+-----------------+-----------------+--------+------------+-----------+------+------------+-----------------+----------+----------+
|_c0|        vehicleId|eventDateTime|gpsLatitude|gpslongitude|gpsAltitude_Last|gpsAltitude_First|            speed|Coasting|Coasting_iso|RashTurning|Idling|OverSpeeding|HarshAcceleration|HarshBrake| eventdate|
+---+-----------------+-------------+-----------+------------+----------------+-----------------+-----------------+--------+------------+-----------+------+------------+-----------------+----------+----------+
|  0|MAT790205M7C08185| 202309100950|  28.869072|  75.9171584|           19298|            19305|              0.0|       0|           0|          0|     0|           0|                0|         0|2023-09-10|
|  1|MAT819003M1A01810| 202309101350| 27.7615936|  77.4720512|           18321|            18483|              0.0|       0|           0|          0|     0|    

24/02/14 08:57:53 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , vehicleId, eventDateTime, gpsLatitude, gpslongitude, gpsAltitude_Last, gpsAltitude_First, speed, Coasting, Coasting_iso, RashTurning, Idling, OverSpeeding, HarshAcceleration, HarshBrake, eventdate
 Schema: _c0, vehicleId, eventDateTime, gpsLatitude, gpslongitude, gpsAltitude_Last, gpsAltitude_First, speed, Coasting, Coasting_iso, RashTurning, Idling, OverSpeeding, HarshAcceleration, HarshBrake, eventdate
Expected: _c0 but found: 
CSV file: file:///kaggle/input/telemetrysample/10min_BEE_GEE_data.csv


In [6]:
# Geohash 
udf1 = F.udf(lambda x,y: pgh.encode(x,y,precision=7)) # Check counts with 5 as well
df1= df.select('vehicleId','eventDateTime','gpsLatitude','gpslongitude',
                'rashturning','overspeeding','harshacceleration','harshbrake',
                udf1('gpsLatitude','gpslongitude').alias('geohash_7'))

In [7]:
udf2 = F.udf(lambda x,y: pgh.encode(x,y,precision=5)) # Check counts with 5 as well
df1= df1.select('vehicleId','eventDateTime','gpsLatitude','gpslongitude',
                'rashturning','overspeeding','harshacceleration','harshbrake','geohash_7',
                udf2('gpsLatitude','gpslongitude').alias('geohash_5'))

In [8]:
df1.show()

+-----------------+-------------+-----------+------------+-----------+------------+-----------------+----------+---------+---------+
|        vehicleId|eventDateTime|gpsLatitude|gpslongitude|rashturning|overspeeding|harshacceleration|harshbrake|geohash_7|geohash_5|
+-----------------+-------------+-----------+------------+-----------+------------+-----------------+----------+---------+---------+
|MAT790205M7C08185| 202309100950|  28.869072|  75.9171584|          0|           0|                0|         0|  ttjupx3|    ttjup|
|MAT819003M1A01810| 202309101350| 27.7615936|  77.4720512|          0|           0|                0|         0|  tszjfvv|    tszjf|
|MAT819003M1A01810| 202309101150| 27.7615456|  77.4720128|          0|           0|                0|         0|  tszjfvv|    tszjf|
|MAT790205M7C08185| 202309101600| 29.2670176|  76.1980224|          0|           0|                0|         0|  ttnnmzv|    ttnnm|
|MAT819003M1A01810| 202309101820| 27.7615936|  77.4720704|          0

In [9]:
# UDF to decode geohash_7
import pygeohash as pgh2
from pyspark.sql.functions import pandas_udf, col
from pyspark.sql.types import ArrayType, DoubleType,FloatType
udf2 = F.udf(lambda x: pgh2.decode(x),ArrayType(FloatType()))

In [10]:
df2 = df1.select('vehicleId','eventDateTime','gpsLatitude','gpslongitude',
'rashturning','overspeeding','harshacceleration','harshbrake','geohash_7',
'geohash_5',udf2('geohash_7').alias('decodedValue'))

In [11]:
from pyspark.sql.functions import split
from pyspark.sql.functions import col
# split the decoded value into latitude and longitude columns
df3 = df2.withColumn("Latitude", col("decodedValue")[0].cast("float"))
df3 = df3.withColumn("Longitude", col("decodedValue")[1].cast("float"))
df3.show(3)

+-----------------+-------------+-----------+------------+-----------+------------+-----------------+----------+---------+---------+--------------+--------+---------+
|        vehicleId|eventDateTime|gpsLatitude|gpslongitude|rashturning|overspeeding|harshacceleration|harshbrake|geohash_7|geohash_5|  decodedValue|Latitude|Longitude|
+-----------------+-------------+-----------+------------+-----------+------------+-----------------+----------+---------+---------+--------------+--------+---------+
|MAT790205M7C08185| 202309100950|  28.869072|  75.9171584|          0|           0|                0|         0|  ttjupx3|    ttjup|[28.87, 75.92]|   28.87|    75.92|
|MAT819003M1A01810| 202309101350| 27.7615936|  77.4720512|          0|           0|                0|         0|  tszjfvv|    tszjf|[27.76, 77.47]|   27.76|    77.47|
|MAT819003M1A01810| 202309101150| 27.7615456|  77.4720128|          0|           0|                0|         0|  tszjfvv|    tszjf|[27.76, 77.47]|   27.76|    77.47

In [12]:
from pyspark.sql.functions import col, unix_timestamp
from pyspark.sql.types import TimestampType

# Assuming df3 is your DataFrame
df3_with_datetime = df3.withColumn("eventDateTime", 
                                   unix_timestamp(col("eventDateTime").cast("string"), 
                                                  "yyyyMMddHHmm").cast(TimestampType()))

df3_with_datetime.show(truncate=False)



+-----------------+-------------------+-----------+------------+-----------+------------+-----------------+----------+---------+---------+--------------+--------+---------+
|vehicleId        |eventDateTime      |gpsLatitude|gpslongitude|rashturning|overspeeding|harshacceleration|harshbrake|geohash_7|geohash_5|decodedValue  |Latitude|Longitude|
+-----------------+-------------------+-----------+------------+-----------+------------+-----------------+----------+---------+---------+--------------+--------+---------+
|MAT790205M7C08185|2023-09-10 09:50:00|28.869072  |75.9171584  |0          |0           |0                |0         |ttjupx3  |ttjup    |[28.87, 75.92]|28.87   |75.92    |
|MAT819003M1A01810|2023-09-10 13:50:00|27.7615936 |77.4720512  |0          |0           |0                |0         |tszjfvv  |tszjf    |[27.76, 77.47]|27.76   |77.47    |
|MAT819003M1A01810|2023-09-10 11:50:00|27.7615456 |77.4720128  |0          |0           |0                |0         |tszjfvv  |tszjf  

In [13]:
df4= df3_with_datetime.drop("decodedVal")
df4 = df4.withColumn("coordinates", concat(col("Latitude"), lit(", "), col("Longitude")))
df4.show(3)

+-----------------+-------------------+-----------+------------+-----------+------------+-----------------+----------+---------+---------+--------------+--------+---------+------------+
|        vehicleId|      eventDateTime|gpsLatitude|gpslongitude|rashturning|overspeeding|harshacceleration|harshbrake|geohash_7|geohash_5|  decodedValue|Latitude|Longitude| coordinates|
+-----------------+-------------------+-----------+------------+-----------+------------+-----------------+----------+---------+---------+--------------+--------+---------+------------+
|MAT790205M7C08185|2023-09-10 09:50:00|  28.869072|  75.9171584|          0|           0|                0|         0|  ttjupx3|    ttjup|[28.87, 75.92]|   28.87|    75.92|28.87, 75.92|
|MAT819003M1A01810|2023-09-10 13:50:00| 27.7615936|  77.4720512|          0|           0|                0|         0|  tszjfvv|    tszjf|[27.76, 77.47]|   27.76|    77.47|27.76, 77.47|
|MAT819003M1A01810|2023-09-10 11:50:00| 27.7615456|  77.4720128|      

In [14]:
df4.groupBy("geohash_7").agg(countDistinct("vehicleId").alias("Distinct_Vehicle_Count")).sort(desc('Distinct_Vehicle_Count')).show()

+---------+----------------------+
|geohash_7|Distinct_Vehicle_Count|
+---------+----------------------+
|  zzzzzzz|                    69|
|  ttw9y98|                    31|
|  ttw9wxv|                    31|
|  ttw9y8j|                    28|
|  ttqz80x|                    25|
|  ttqz81w|                    24|
|  ttqz80w|                    24|
|  ttqz81y|                    24|
|  ttw9y92|                    21|
|  ttqz84j|                    19|
|  ttqz84w|                    18|
|  ttqz81n|                    18|
|  ttqz2pc|                    17|
|  ttw9wxw|                    17|
|  ttqz8r9|                    17|
|  ttqxqcc|                    17|
|  ttqxrvv|                    16|
|  ttqz2p6|                    16|
|  ttqz81m|                    16|
|  ttqwvgd|                    15|
+---------+----------------------+
only showing top 20 rows



In [15]:
unique_count = df4.agg(countDistinct('vehicleId').alias('nunique_vehicleId')).collect()[0]['nunique_vehicleId']

In [16]:
unique_count

1350

In [17]:
unique_vehicleIds = df4.select('vehicleId').distinct().rdd.map(lambda row: row['vehicleId']).collect()

#print("Unique Vehicle IDs:", unique_vehicleIds)

In [18]:
# Fleet Asset Data 

In [19]:
asset = spark.read.csv("/kaggle/input/fleets/Fleet.csv",header=True)

In [20]:
tml_asset = asset

In [21]:
tml_asset= tml_asset.withColumn("asset_number", trim(tml_asset["asset_number"]))
tml_asset_cleaned = tml_asset.withColumn("asset_number", trim(tml_asset["asset_number"]))
# Joining DataFrames based on 'vehicleId' and 'asset_number'
df4_with_asset = df4.join(tml_asset_cleaned, df4.vehicleId == tml_asset_cleaned.asset_number, "left")

In [22]:
df4_with_asset.show()

24/02/14 08:58:07 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/02/14 08:58:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , asset_number, first_sale_dt, LOB, PPL, PL, VC, DEALER_NAME, DEALER_CODE, dlr_region, dlr_area, DEALER_STATE, dlr_org_city, dlr_district
 Schema: _c0, asset_number, first_sale_dt, LOB, PPL, PL, VC, DEALER_NAME, DEALER_CODE, dlr_region, dlr_area, DEALER_STATE, dlr_org_city, dlr_district
Expected: _c0 but found: 
CSV file: file:///kaggle/input/fleets/Fleet.csv


+-----------------+-------------------+-----------+------------+-----------+------------+-----------------+----------+---------+---------+--------------+--------+---------+------------+----+------------+-------------+----+----+----+----+-----------+-----------+----------+--------+------------+------------+------------+
|        vehicleId|      eventDateTime|gpsLatitude|gpslongitude|rashturning|overspeeding|harshacceleration|harshbrake|geohash_7|geohash_5|  decodedValue|Latitude|Longitude| coordinates| _c0|asset_number|first_sale_dt| LOB| PPL|  PL|  VC|DEALER_NAME|DEALER_CODE|dlr_region|dlr_area|DEALER_STATE|dlr_org_city|dlr_district|
+-----------------+-------------------+-----------+------------+-----------+------------+-----------------+----------+---------+---------+--------------+--------+---------+------------+----+------------+-------------+----+----+----+----+-----------+-----------+----------+--------+------------+------------+------------+
|MAT790205M7C08185|2023-09-10 09:50:0

In [23]:
final_df = df4_with_asset.filter((df4_with_asset.geohash_7 != 'zzzzzzz') & (df4_with_asset.geohash_7 != '7zzzzzz'))

In [24]:
final_df=df4_with_asset.select('vehicleId','geohash_7','geohash_5','Latitude', 'Longitude','eventDateTime','LOB','PL','PPL','rashturning','overspeeding','harshacceleration','harshbrake').groupBy('geohash_7','geohash_5','vehicleId','Latitude', 'Longitude','eventDateTime','LOB','PL','PPL','rashturning','overspeeding','harshacceleration','harshbrake').agg(countDistinct('vehicleId'))


In [25]:
final_df=final_df.withColumnRenamed('count(DISTINCT vehicleId)', 'VinCount')

In [26]:
final_df.columns

['geohash_7',
 'geohash_5',
 'vehicleId',
 'Latitude',
 'Longitude',
 'eventDateTime',
 'LOB',
 'PL',
 'PPL',
 'rashturning',
 'overspeeding',
 'harshacceleration',
 'harshbrake',
 'VinCount']

In [27]:
df5 = final_df

In [28]:
from pyspark.sql.functions import col, sum as _sum
# Filter the original DataFrame to create four separate DataFrames
rashturning_df = df5.filter((col("rashturning") > 0) & (col("overspeeding") == 0) & (col("harshacceleration") == 0) & (col("harshbrake") == 0))
overspeeding_df = df5.filter((col("rashturning") == 0) & (col("overspeeding") > 0) & (col("harshacceleration") == 0) & (col("harshbrake") == 0))
harshacceleration_df = df5.filter((col("rashturning") == 0) & (col("overspeeding") == 0) & (col("harshacceleration") > 0) & (col("harshbrake") == 0))
harshbrake_df = df5.filter((col("rashturning") == 0) & (col("overspeeding") == 0) & (col("harshacceleration") == 0) & (col("harshbrake") > 0))

# Aggregate VinCount in a temporary DataFrame
vincount_df = df5.groupBy("vehicleID").agg(_sum("VinCount").alias("totalVinCount"))
vincount_df = vincount_df.filter(col("totalVinCount") > 10)

# Join the filtered DataFrames with the aggregated VinCount DataFrame
rashturning_df = rashturning_df.join(vincount_df, "vehicleID")
overspeeding_df = overspeeding_df.join(vincount_df, "vehicleID")
harshacceleration_df = harshacceleration_df.join(vincount_df, "vehicleID")
harshbrake_df = harshbrake_df.join(vincount_df, "vehicleID")

In [29]:
from pyspark.sql import DataFrame

# Combine all filtered DataFrames into one master DataFrame
master_df = rashturning_df.union(overspeeding_df)\
    .union(harshacceleration_df)\
    .union(harshbrake_df)

# Drop duplicate rows if necessary
master_df = master_df.dropDuplicates()

#
master_df.count()


1856

In [30]:
master_df.show()

+-----------------+---------+---------+--------+---------+-------------------+----------+------------------+--------------+-----------+------------+-----------------+----------+--------+-------------+
|        vehicleId|geohash_7|geohash_5|Latitude|Longitude|      eventDateTime|       LOB|                PL|           PPL|rashturning|overspeeding|harshacceleration|harshbrake|VinCount|totalVinCount|
+-----------------+---------+---------+--------+---------+-------------------+----------+------------------+--------------+-----------+------------+-----------------+----------+--------+-------------+
|MAT790212N2C05073|  ttrjq6s|    ttrjq|   30.47|    77.62|2023-09-10 08:20:00|MCV Trucks|          LPT 1918|    MCV LPT 19|          0|           0|                1|         0|       1|           38|
|MAT790219N7J22586|  teyjgf4|    teyjg|   22.12|    76.11|2023-09-10 10:20:00|MCV Trucks|         ULTRA1918|    MCV LPT 19|          0|           0|                1|         0|       1|          

# cKDTree

In [31]:
world_cities = spark.read.csv("/kaggle/input/worldcities-fortelemetry/worldcities.csv",header=True,inferSchema=True)

In [32]:
world_cities = world_cities.filter(col("country") == 'India')

In [33]:
world_cities.count()

1600

In [34]:
world_cities.show()

+--------------+--------------+-------+-------+-------+----+----+--------------+-------+----------+----------+
|          city|    city_ascii|    lat|    lng|country|iso2|iso3|    admin_name|capital|population|        id|
+--------------+--------------+-------+-------+-------+----+----+--------------+-------+----------+----------+
|         Delhi|         Delhi|28.6667|77.2167|  India|  IN| IND|         Delhi|  admin|   3.187E7|1356872604|
|        Mumbai|        Mumbai|19.0758|72.8775|  India|  IN| IND|   Mahārāshtra|  admin|  2.2186E7|1356226629|
|       Kolkāta|       Kolkata|22.5727|88.3639|  India|  IN| IND|   West Bengal|  admin|  1.8698E7|1356060520|
|     Bangalore|     Bangalore|12.9791|77.5913|  India|  IN| IND|     Karnātaka|  admin|  1.3999E7|1356410365|
|       Chennai|       Chennai|13.0825| 80.275|  India|  IN| IND|    Tamil Nādu|  admin|  1.1564E7|1356374944|
|     Hyderābād|     Hyderabad|17.3617|78.4747|  India|  IN| IND|     Telangana|  admin| 9840000.0|1356871768|
|

In [35]:
def create_geometry(lat, lng):
    return Point(float(lng), float(lat))

create_geometry_udf = udf(create_geometry)

In [36]:
df_ind = world_cities.select("city_ascii", "lat", "lng")

In [37]:
df_ind.show()

+--------------+-------+-------+
|    city_ascii|    lat|    lng|
+--------------+-------+-------+
|         Delhi|28.6667|77.2167|
|        Mumbai|19.0758|72.8775|
|       Kolkata|22.5727|88.3639|
|     Bangalore|12.9791|77.5913|
|       Chennai|13.0825| 80.275|
|     Hyderabad|17.3617|78.4747|
|          Pune|18.5196|73.8553|
|     Ahmedabad|  23.03|  72.58|
|     Allahabad|  25.45|  81.85|
|         Surat|  21.17|  72.83|
|       Lucknow| 26.847| 80.947|
|        Jaipur|26.9167|75.8667|
|      Cawnpore|26.4725|80.3311|
|      Mirzapur|  25.15|  82.58|
|        Nagpur|21.1539|79.0831|
|     Ghaziabad|28.6667|77.4167|
|      Vadodara|   22.3|   73.2|
|Vishakhapatnam|17.7333|83.3167|
|        Indore|22.7206|75.8472|
|         Thane|  19.18|72.9633|
+--------------+-------+-------+
only showing top 20 rows



In [38]:
from pyspark.sql.functions import col
from pyspark.sql import Row
from pyspark.sql import SparkSession
from scipy.spatial import cKDTree

# Collect the latitude, longitude, and city_ascii columns from df2
df2_coords = df_ind.select("lat", "lng", "city_ascii").collect()

# Create KDTree for df2 coordinates
kdtree = cKDTree([(row["lat"], row["lng"]) for row in df2_coords])

In [39]:
def find_nearest_city(row):
    lat = row["Latitude"]
    lng = row["Longitude"]
    # Query the KDTree to find the index of the nearest city in df2
    index = kdtree.query([lat, lng])[1]
    # Get the corresponding city name from df2_coords
    city_name = df2_coords[index]["city_ascii"]
    return city_name

# Convert df1 to RDD of Rows and apply the find_nearest_city function
df1_rdd = master_df.rdd.map(lambda row: Row(nearest_city=find_nearest_city(row), **row.asDict()))

# Create a new DataFrame from the RDD
df1_with_nearest_city = df1_rdd.toDF()

# Show or perform further operations on df1_with_nearest_city
df1_with_nearest_city.show()

+------------+-----------------+---------+---------+------------------+-----------------+-------------------+----------+------------------+--------------+-----------+------------+-----------------+----------+--------+-------------+
|nearest_city|        vehicleId|geohash_7|geohash_5|          Latitude|        Longitude|      eventDateTime|       LOB|                PL|           PPL|rashturning|overspeeding|harshacceleration|harshbrake|VinCount|totalVinCount|
+------------+-----------------+---------+---------+------------------+-----------------+-------------------+----------+------------------+--------------+-----------+------------+-----------------+----------+--------+-------------+
|    Bilaspur|MAT790212N2C05073|  ttrjq6s|    ttrjq|30.469999313354492|77.62000274658203|2023-09-10 08:20:00|MCV Trucks|          LPT 1918|    MCV LPT 19|          0|           0|                1|         0|       1|           38|
|      Indore|MAT790219N7J22586|  teyjgf4|    teyjg|  22.1200008392334|7

In [40]:
df_Jan = df1_with_nearest_city.withColumnRenamed("nearest_city", "City")

In [41]:
df_Jan.show()

+----------+-----------------+---------+---------+------------------+-----------------+-------------------+----------+------------------+--------------+-----------+------------+-----------------+----------+--------+-------------+
|      City|        vehicleId|geohash_7|geohash_5|          Latitude|        Longitude|      eventDateTime|       LOB|                PL|           PPL|rashturning|overspeeding|harshacceleration|harshbrake|VinCount|totalVinCount|
+----------+-----------------+---------+---------+------------------+-----------------+-------------------+----------+------------------+--------------+-----------+------------+-----------------+----------+--------+-------------+
|  Bilaspur|MAT790212N2C05073|  ttrjq6s|    ttrjq|30.469999313354492|77.62000274658203|2023-09-10 08:20:00|MCV Trucks|          LPT 1918|    MCV LPT 19|          0|           0|                1|         0|       1|           38|
|    Indore|MAT790219N7J22586|  teyjgf4|    teyjg|  22.1200008392334|76.11000061